In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import lxml
from urllib.request import Request, urlopen

### URLs Used

In [2]:
WINE_REGION_URL2 = "https://www.tonymappedit.com/top-10-wine-producing-countries/"
WIKI_COUNTRIES = "https://en.wikipedia.org/wiki/ISO_3166-1_alpha-3"
VINEYARD_URL = "https://www.bkwine.com/features/more/world-wine-production-reaches-record-level-2018-consumption-stable/"
EXPORT_UL = "http://www.worldstopexports.com/wine-exports-country/"
IMPORT_URL = "http://www.worldstopexports.com/top-wine-importing-countries/"
CONSUMPTION_URL = ""

### Wine Production Data

In [3]:
'''scrape wine region data with coresponding wines'''
req = Request(WINE_REGION_URL2 , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

# html_text = requests.get(WINE_REGION_URL).text
soup = BeautifulSoup(webpage, "html.parser")

In [4]:
country1 = []
countries = soup.find_all("ol")[1]
items = countries.find_all("li")

for country in items:
    country1.append(country.text.split(": "))

In [5]:
country2 = []
countries2 = soup.find_all("ol")[2]
items = countries2.find_all("li")

for country in items:
    country2.append(country.text.split(": "))

In [6]:
country_production = country1 + country2
# country_production

In [7]:
country_production_df = pd.DataFrame(country_production).rename(columns = {0: "Country", 1: "WineProduction[HCL]"}).set_index("Country")
# country_production_df

### Country Code Data

In [8]:
'''scrape wine region data with coresponding wines'''
req = Request(WIKI_COUNTRIES , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

# html_text = requests.get(WINE_REGION_URL).text
soup = BeautifulSoup(webpage, "html.parser")

In [9]:
table = soup.find("div", class_="plainlist")
country_codes = table.find_all("span")
countries = table.find_all("a")

In [10]:
country_list = []
for country in countries:
    country_list.append(country.text)

In [11]:
code_list = []
for code in country_codes:
    code_list.append(code.text)

In [12]:
#Province of China was index 229. 
country_df = pd.DataFrame(country_list).drop([11, 229]).reset_index().drop(columns = ["index"])
country_df = country_df.rename(columns = {0: "Country"}).reset_index().set_index("index")

In [13]:
code_df = pd.DataFrame(code_list).drop([11]).reset_index().drop(columns = ["index"])
code_df = code_df.rename(columns = {0: "CODES"}).reset_index().set_index("index")

In [14]:
country_code_df = pd.merge(country_df, code_df, on="index", how="outer").set_index("Country")
# country_code_df

In [15]:
wine_production_HCL = pd.merge(country_production_df,country_code_df,  how="right", on="Country").rename(columns = {'Alpha-3 code':'CODE'}).fillna(0)
# wine_production_HCL

In [16]:
# wine_production_HCL.to_csv("wine_production.csv")

### Vineyard Data

In [17]:
'''scrape wine region data with coresponding wines'''
req = Request(VINEYARD_URL , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()

# html_text = requests.get(WINE_REGION_URL).text
soup = BeautifulSoup(webpage, "html.parser")

In [18]:
vineyard_table = soup.find_all("table", id="tablepress-762")[0]
vine_country = vineyard_table.find_all("td", class_="column-2")
vine_acreage = vineyard_table.find_all("td", class_="column-3")

In [19]:
vine_countries = []
for country in vine_country:
    vine_countries.append(country.text)

In [20]:
plot_sizes = []
for plot_size in vine_acreage:
    plot_sizes.append(plot_size.text)

In [21]:
country_vy = pd.DataFrame(vine_countries).drop([0,27,28,29,30]).reset_index().replace('\*','',regex=True).rename(columns = {0: "Country"})
size_vy = pd.DataFrame(plot_sizes).drop([0,27,28]).reset_index().rename(columns = {0: "Largest Vineyards"})

In [22]:
vineyard_data = pd.merge(country_vy, size_vy, on="index").drop(columns = ["index"]).set_index("Country")
# vineyard_data

In [23]:
world_wine_vy = pd.merge(wine_production_HCL, vineyard_data, on="Country", how="left").fillna(0)
world_wine_vy

,WineProduction[HCL],CODES,Largest Vineyards
Country,,,
Italy,54.8,ITA,702
France,49.1,FRA,789
Spain,44.4,ESP,969
Argentina,14.5,ARG,219
Chile,12.9,CHL,212
...,...,...,...
Wallis and Futuna,0,WLF,0
Samoa,0,WSM,0
Yemen,0,YEM,0


### Export Data

### Import Data

### Consumption Data

In [24]:
world_wine_vy.to_csv("world_wine_data.csv")